PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=2a4a3dae42db343ee1858346fd4deeafb2688423974c399288b3a95ec47ce449
  Stored in directory: /root/.cache/pip/wheels/97/f5/c0/947e2c0942b361ffe58651f36bd7f13772675b3863fd63d1b1
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.4
    Uninstalling pyspark-3.5.4:
      Successfully uninstalled pyspark-3.5.4


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

# 보스턴 주택 가격 예측 모델 만들기




In [ ]:
spark

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2025-02-11 08:12:09--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.193.122, 52.92.251.2, 52.218.233.25, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.193.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.04s   

2025-02-11 08:12:09 (861 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [ ]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Feb  7 14:20 sample_data
-rw-r--r-- 1 root root 36240 Jan 31  2021 boston_housing.csv


In [ ]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [ ]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [ ]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [ ]:
data_2 = assembler.transform(data)

In [ ]:
data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [ ]:
train, test = data_2.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

## 모델 성능 측정

In [ ]:
evaluation_summary = model.evaluate(test)

In [ ]:
evaluation_summary

In [ ]:
evaluation_summary.meanAbsoluteError

3.4964886201036176

In [ ]:
evaluation_summary.rootMeanSquaredError

5.186649547151137

In [ ]:
evaluation_summary.r2

0.7127737873826929

## 모델 예측값 살펴보기

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.00632|18.0| 2.31|   0| 0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|29.791798003993595|
|0.01381|80.0| 0.46|   0| 0.422|7.875|32.0|5.6484|  4|255|   14.4|394.23| 2.97|50.0|[0.01381,80.0,0.4...|41.050701617605114|
|0.01439|60.0| 2.93|   0| 0.401|6.604|18.8|6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...|31.421840522581984|
| 0.0187|85.0| 4.15|   0| 0.429|6.516|27.7|8.5353|  4|351|   17.9|392.43| 6.36|23.1|[0.0187,85.0,4.15...| 25.56737630420656|
|0.01965|80.0| 1.76|   0| 0.385| 6.23|31.5|9.0892|  1|241|   18.2| 341.6|12.93|20.1|[0.01965,80.0,1.7...| 20.74591465318253|


In [ ]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|24.0|[0.00632,18.0,2.3...|29.791798003993595|
|50.0|[0.01381,80.0,0.4...|41.050701617605114|
|29.1|[0.01439,60.0,2.9...|31.421840522581984|
|23.1|[0.0187,85.0,4.15...| 25.56737630420656|
|20.1|[0.01965,80.0,1.7...| 20.74591465318253|
|16.5|[0.02498,0.0,1.89...|22.052593323314472|
|34.7|[0.02729,0.0,7.07...|30.808747059681018|
|33.4|[0.03237,0.0,2.18...|28.857673154093504|
|34.9|[0.03359,75.0,2.9...| 34.32804685189889|
|48.5|[0.0351,95.0,2.68...|  42.3029544316986|
|23.5|[0.03584,80.0,3.3...| 30.12160923816731|
|27.9|[0.03615,80.0,4.9...|31.983549493483466|
|20.7|[0.03738,0.0,5.19...| 21.90174815673657|
|33.3|[0.04011,80.0,1.5...| 36.08184881090703|
|22.9|[0.04203,28.0,15....| 29.00092083751408|
|20.6|[0.04294,28.0,15....|27.180559429232343|
|20.5|[0.04337,21.0,5.6...|24.117968762446534|
|11.9|[0.04741,0.0,11.9...|22.478819870669426|
|21.9|[0.0481

In [ ]:
model.save("boston_housing_model")

In [ ]:
!ls boston_housing_model

data  metadata


In [ ]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Feb 11 08:20 data
drwxr-xr-x 2 root root 4096 Feb 11 08:20 metadata


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model"
model.save(path)

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [ ]:
predictions2 = loaded_model.transform(test)

In [ ]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|24.0|[0.00632,18.0,2.3...|29.791798003993595|
|50.0|[0.01381,80.0,0.4...|41.050701617605114|
|29.1|[0.01439,60.0,2.9...|31.421840522581984|
|23.1|[0.0187,85.0,4.15...| 25.56737630420656|
|20.1|[0.01965,80.0,1.7...| 20.74591465318253|
|16.5|[0.02498,0.0,1.89...|22.052593323314472|
|34.7|[0.02729,0.0,7.07...|30.808747059681018|
|33.4|[0.03237,0.0,2.18...|28.857673154093504|
|34.9|[0.03359,75.0,2.9...| 34.32804685189889|
|48.5|[0.0351,95.0,2.68...|  42.3029544316986|
|23.5|[0.03584,80.0,3.3...| 30.12160923816731|
|27.9|[0.03615,80.0,4.9...|31.983549493483466|
|20.7|[0.03738,0.0,5.19...| 21.90174815673657|
|33.3|[0.04011,80.0,1.5...| 36.08184881090703|
|22.9|[0.04203,28.0,15....| 29.00092083751408|
|20.6|[0.04294,28.0,15....|27.180559429232343|
|20.5|[0.04337,21.0,5.6...|24.117968762446534|
|11.9|[0.04741,0.0,11.9...|22.478819870669426|
|21.9|[0.0481